In [22]:
import pandas as pd
import numpy as np

In [23]:
df = pd.read_csv('https://github.com/pvateekul/2110446_DSDE_2023s2/raw/main/code/Week03_ML/mushroom2020_dataset.csv')
df.head()

,id,label,cap-shape,cap-surface,bruises,odor,gill-attachment,gill-spacing,gill-size,stalk-shape,...,ring-number,ring-type,spore-print-color,population,habitat,cap-color-rate,gill-color-rate,veil-color-rate,stalk-color-above-ring-rate,stalk-color-below-ring-rate
0,1,p,x,s,t,p,f,c,n,e,...,o,p,k,s,u,1.0,3.0,1.0,1.0,1.0
1,2,e,x,s,t,a,f,c,b,e,...,o,p,n,n,g,2.0,3.0,1.0,1.0,1.0
2,3,e,b,s,t,l,f,c,b,e,...,o,p,n,n,m,3.0,1.0,1.0,1.0,1.0
3,4,p,x,y,t,p,f,c,n,e,...,o,p,k,s,u,3.0,1.0,1.0,1.0,1.0
4,5,e,x,s,f,n,f,w,b,t,...,o,e,n,a,g,4.0,3.0,1.0,1.0,1.0


In [24]:
df['gill-size'].isna().sum()

121

In [25]:
df.shape

(5824, 24)

what is edible wa

In [26]:
# 2. Drop rows where the target (label) variable is missing.
df.dropna(subset=['label'], inplace=True)

In [27]:
# 3. Drop the following variables:
df.drop(columns=['id','gill-attachment','gill-spacing', 'gill-size','gill-color-rate',
'stalk-root', 'stalk-surface-above-ring', 'stalk-surface-below-ring', 'stalk-color-above-ring-rate','stalk-color-below-ring-rate','veil-color-rate','veil-type'],
inplace=True)
df.shape

(5764, 12)

In [28]:
df.isnull().sum()

label                  0
cap-shape              0
cap-surface           27
bruises               99
odor                  99
stalk-shape          121
ring-number           62
ring-type             62
spore-print-color     56
population            56
habitat               31
cap-color-rate        27
dtype: int64

In [29]:
# 4. Examine the number of rows, the number of digits, and whether any are missing.
df.shape

(5764, 12)

In [30]:

# 5. Fill missing values by adding the mean for numeric variables and the mode for nominal variables.

nominal_variables=[  "label", 
                     "cap-shape", 
                     "cap-surface",
                     "bruises",
                     "odor", 
                     "stalk-shape", 
                     "ring-number", 
                     "ring-type", 
                     "spore-print-color",
                     "population", 
                     "habitat"]
numeric_variables=["cap-color-rate"]

mode=df[nominal_variables].mode().iloc[0]
df.fillna(mode, inplace=True)
mean=df.mean(axis=0, skipna=True, numeric_only=True)
df.fillna(mean, inplace=True)
df.isnull().sum()

label                0
cap-shape            0
cap-surface          0
bruises              0
odor                 0
stalk-shape          0
ring-number          0
ring-type            0
spore-print-color    0
population           0
habitat              0
cap-color-rate       0
dtype: int64

In [31]:
# 6. Convert the label variable e (edible) to 1 and p (poisonous) to 0 and check the quantity. class0: class1
df.label = df.label.astype('string')
df.label = df.label.map({'e': 1, 'p': 0})
print(df.label.sum())
print(len(df.label)-df.label.sum())
df.label.value_counts()

2104
3660


label
0    3660
1    2104
Name: count, dtype: int64

In [32]:
# 7. Convert the nominal variable to numeric using a dummy code with drop_first = True.
dummy_df = pd.get_dummies(df[nominal_variables[1:]], drop_first=True)
df = pd.concat([df,dummy_df], axis=1)
df.drop(nominal_variables[1:], axis=1,inplace=True)

In [21]:
# 8. Split train/test with 20% test, stratify, and seed = 2020.
from sklearn.model_selection import train_test_split
x=df.copy()
y=x.pop("label")
x_train,x_test,y_train,y_test = train_test_split(x,y,stratify=y,test_size=0.20, random_state=2020)
print(len(x_train))
print(len(x_test))

4611
1153


In [ ]:
from sklearn.model_selection import GridSearchCV
from sklearn.ensemble import RandomForestClassifier
param_grid={
    'criterion': ['gini','entropy'],
    'max_depth': [2,3,6],
    'min_samples_leaf':[2,5,10],
    'n_estimators':[100,200],
    'random_state': [2020]
}
rfc=RandomForestClassifier()
clf = GridSearchCV(estimator=rfc, param_grid=param_grid, cv= 5)
clf.fit(x_train, y_train)

GridSearchCV(cv=5, estimator=RandomForestClassifier(),
             param_grid={'criterion': ['gini', 'entropy'],
                         'max_depth': [2, 3, 6], 'min_samples_leaf': [2, 5, 10],
                         'n_estimators': [100, 200], 'random_state': [2020]})

In [ ]:
# 10. Predict the testing data set with confusion_matrix and classification_report.)
from sklearn.metrics import classification_report, confusion_matrix
y_predict = clf.predict(x_test)
print(classification_report(y_test, y_predict))
print(confusion_matrix(y_test, y_predict))

              precision    recall  f1-score   support

           0       0.99      1.00      1.00       732
           1       1.00      0.99      0.99       421

    accuracy                           0.99      1153
   macro avg       1.00      0.99      0.99      1153
weighted avg       0.99      0.99      0.99      1153

[[731   1]
 [  5 416]]


In [ ]:
clf.best_params_

{'criterion': 'gini',
 'max_depth': 6,
 'min_samples_leaf': 2,
 'n_estimators': 100,
 'random_state': 2020}